# Node Classification via RSFs 

In this notebook, the RSF based methods are illustrated in the context of node classification problem. 

$\underline{\text{Problem}}$ : Given a graph $\mathcal{G}=(\mathcal{V},\mathcal{E},w)$ and a few labels on a vertex subset $\ell\subset\mathcal{V}$, node classification task is to infer the labels or classes of the other vertices $u=\mathcal{V}\setminus\ell$. 

$\underline{\text{Label Encoding}}$ : To able to use the priory information in the algorithms, the known labels are interpreted by the following binary encoding for each class $c$: 

$$
\hspace{5cm} \forall i \in \mathcal{V}, \quad \mathbf{y}_c(i) = \begin{cases}
1 \text{ if } i \text{ is known to belong to the } c\text{-th class} \\ 
0 \text{ otherwise}
\end{cases}
$$ 
which defines the columns of the encoding matrix $\mathsf{Y}=[\mathbf{y}_1,\mathbf{y}_2,...,\mathbf{y}_c]\in\mathbb{R}^{|\mathcal{V}|\times C}$ for $C$ classes.


$\underline{\text{Solutions}}$ : Given this encoding, two well-known baseline approaches to solve the problem are: 
- Label Propagatation ([Zhu et.al.](http://pages.cs.wisc.edu/~jerryzhu/pub/thesis.pdf))
- Generalized Optimization Framework for Graph-based Semi-supervised Learning ([Avrachenkov et.al.](https://arxiv.org/abs/1110.4278))

In the following, we illustrate the RSF versions of these algorithms.

In [1]:
### PACKAGES AND FUNCTIONS ### 
using RandomForests
using LightGraphs
using GraphIO
using Plots
using Plots.PlotMeasures
using StatsBase
using Clustering
using SparseArrays
using LinearAlgebra
using MLJ
using JLD2
using LaTeXStrings
using NamedArrays

In [2]:
plotly()
include("utils.jl")
include("ssl_data_loaders.jl")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /home/pilavciy-admin/.julia/packages/ORCA/U5XaN/src/ORCA.jl:8


load_citeseer (generic function with 1 method)

### Load the benchmark dataset (Graph and ground truth labels)


In [3]:
G , gr_truth = load_citeseer();

In [4]:
## Generate the encoding given the labels. 
gr_truth = categorical(gr_truth)
classnum = length(unique(gr_truth))
encoded_labels = encode_labels(gr_truth,classnum)

# The label of 10th vertex and the corresponding encoding
gr_truth[10],encoded_labels[10,:]

(CategoricalArrays.CategoricalValue{Int64,UInt32} 2, [0.0, 1.0, 0.0, 0.0, 0.0, 0.0])

### Priory labels are generated 

In [5]:
m = 30
Y,labeled_nodes = sample_labels(encoded_labels,m)
unlabeled_nodes = setdiff(1:nv(G),labeled_nodes);

## Label Propagation 

Label propagation is a simple algorithm that follows the following procedure:
 1. Initialize $t=0$, $\mathsf{Y}^{(t)}=\mathsf{Y}$ and $P = \mathsf{D}^{-1}\mathsf{W} \in \mathbb{R}^{|\mathcal{V}| \times |\mathcal{V}|}$ where $\mathsf{W}$ is the (weighted) adjacency matrix and $\mathsf{D}$ is the diagonal degree matrix
 2. Compute $\mathsf{Y}^{(t+1)} \leftarrow \mathsf{P}\mathsf{Y}^{(t)}$
 3. Set $\mathsf{Y}^{(t+1)}_{\ell|:} = \mathsf{Y}_{\ell|:}$
 4. Go back to step 2 if $\mathsf{Y}^{(t+1)}$ is not converged. 
 
In the end of this procedure, $\mathsf{Y}^{(t+1)}$ converges to the following for each class $c$: 
$$
\mathsf{F}_{i,c} = \begin{cases}
\mathsf{Y}_{i,c}, &\text{ if } i \in \ell  \\ 
(-(\mathsf{L}_{u|u})^{-1}\mathsf{L}_{u|\ell}\mathsf{Y}_{\ell|:})_{i,c}, &\text{ otherwise }   \\ 
\end{cases}
$$
In the termination of the algorithm, the label of every node $i\in u$ is determined by $\arg\min_{c}\mathsf{F}_{i,c}$

### Label propagation with RSFs. 
- It is also possible to approximate this solution by using random spanning forests. 
- Consider random spanning forests with the pre-determined rootset $\ell$. 
- This means the loop erased random walks in Wilson's algorithm only stops at the nodes in $\ell$. 
- With this setup, the proposed estimator $\tilde{\mathbf{x}}$ becomes an unbiased estimator of $\mathsf{F}$ given the input vectors $\mathsf{Y}$

In [6]:
nrep = 20
rootset = labeled_nodes
est_cl_hat_lp,_ = (ssl_rootset(G,Y,0.0,0.0,method="exact",nrep=nrep,rootset=rootset))
est_cl_tilde_lp,_ = (ssl_rootset(G,Y,0.0,0.0,method="xtilde",nrep=nrep,rootset=rootset));


## Generalized Optimization Framework for Graph-based Semi-supervised Learning

The optimization framework proposed in [Avrachenkov et. al.]() admits the following solution: 
$$
\mathsf{F} = \frac{\mu}{\mu+2}\left(\mathsf{I} - \frac{2}{\mu+2}\mathsf{D}^{-\sigma} \mathsf{W}\mathsf{D}^{\sigma-1}\right)^{-1}\mathsf{Y}
$$
where $\mu$ is the regularization parameter and $\sigma$ choses the regularizer Laplacian. This can be also written as: 

$$ 
\mathsf{F} = \mathsf{D}^{1-\sigma }\mathsf{K} \mathsf{D}^{\sigma -1}\mathsf{Y} \text{ with } \mathsf{K} = (\mathsf{L} + \mathsf{Q})^{-1}\mathsf{Q}
$$
where $\mathsf{Q} = \frac{\mu}{2}\mathsf{D}$. 

### RSF approximation. 
- The computationally expensive part is the product $\mathsf{K} \mathsf{D}^{\sigma -1}\mathsf{Y}$ 
- It can be approximated by the proposed estimators with the input vector $\mathsf{Y}= \mathsf{D}^{\sigma -1}\mathsf{Y}$ and the $q$ values $\mathsf{Q} = \frac{\mu}{2}\mathsf{D}$.
- Moreover, $\mu$ can be automatically selected to get the best classification performance 

### Hyperparameter ($\mu$) selection
$\mu$ is automatically selected using leave-one-out cross validation for each method


In [7]:
loocv_nrep = 30
MU = [0.01,0.1,1.0,10.0]
mu_hat,err_hat = ssl_LOOCV(G,Y,labeled_nodes,mu_range=MU,method="exact")
mu_tilde,err_tilde = ssl_LOOCV(G,Y,labeled_nodes,mu_range=MU,nrep=loocv_nrep,method="xtilde")
mu_bar,err_bar = ssl_LOOCV(G,Y,labeled_nodes,mu_range=MU,nrep=loocv_nrep,method="xbar"); 


plot(MU,err_hat,line=(:solid,6),label="LOOCV Score of ̂x")
plot!(MU,err_tilde,line=(:solid,6),label="LOOCV Score of ̃x")
plot!(MU,err_bar,line=(:solid,6),label="LOOCV Score of ̄x")
plot!(xlabel = "mu", ylabel = "LOOCV Score")

plot!(size=[800,500],tickfont=20, guidefont=20,legendfont=15,legend=:outertopright,leftmargin=20mm)


<!DOCTYPE html>
 
 
 Plots.jl

### Estimation of the results via RSFs

In [8]:
nrep = 200
sigma = 0.0
est_cl_hat, _ = ssl(G,Y,mu_hat,sigma,method="exact",nrep=nrep)
est_cl_tilde, _ = ssl(G,Y,mu_tilde,sigma,method="xtilde",nrep=nrep)
est_cl_bar, _ = ssl(G,Y,mu_bar,sigma,method="xbar",nrep=nrep);

In [9]:
## Calculate the classification accuracy of the results
acc = zeros(2,3)
acc[1,1] =  accuracy(gr_truth[unlabeled_nodes], categorical(est_cl_hat_lp[unlabeled_nodes]))[1]
acc[1,2] =  accuracy(gr_truth[unlabeled_nodes], categorical(est_cl_tilde_lp[unlabeled_nodes]))[1]
acc[1,3] =  NaN
acc[2,1] =  accuracy(gr_truth[unlabeled_nodes], categorical(est_cl_hat[unlabeled_nodes]))[1]
acc[2,2] =  accuracy(gr_truth[unlabeled_nodes], categorical(est_cl_tilde[unlabeled_nodes]))[1]
acc[2,3] =  accuracy(gr_truth[unlabeled_nodes], categorical(est_cl_bar[unlabeled_nodes]))[1];
acc =NamedArray(acc)
setnames!(acc, ["LP","gSSL"], 1)
setnames!(acc, ["xhat","xtilde","xbar"], 2);
setdimnames!(acc, "Solution", 1)
setdimnames!(acc, "Estimator", 2)
println("Classification accuracy of LP, g-SSL and their RSF versions")
display(acc)

2×3 Named Array{Float64,2}
Solution ╲ Estimator │     xhat    xtilde      xbar
─────────────────────┼─────────────────────────────
LP                   │  0.68342  0.676166       NaN
gSSL                 │ 0.695855  0.626943  0.694819

Classification accuracy of LP, g-SSL and their RSF versions
